In [15]:
from dotenv import load_dotenv
import os
from openai import OpenAI, AsyncOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel

load_dotenv(override=True)

class Psycho_Profiler:
    INSTRUCTIONS = """
    You are agent creating structured psychological profiles for testing psychoterapeutical app.
    follow given structure

    {
    "profile_nickname": "",
    "basic_identification": {
        "age": "",
        "gender": "",
        "marital_status": "",
        "occupation": "",
        "education": "",
        "living_environment": ""
    },
    "psychological_history": {
        "childhood": "",
        "parenting_style": "",
        "childhood_emotions": [],
        "developmental_trauma": ""
    },
    "current_issues": {
        "somatization": "",
        "stress": "",
        "emotional_state": [],
        "behavior": "",
        "reality_interpretation": ""
    },
    "personality_traits": {
        "neuroticism": "",
        "agreeableness": "",
        "extraversion": "",
        "conscientiousness": "",
        "openness_to_change": ""
    },
    "cognitive_schemas": {
        "core_beliefs": [
        "",
        ""
        ],
        "automatic_thought": "",
        "behavioral_consequence": "",
        "shift_in_therapy": ""
    }
    }

    Strict requirements:
    - **Do not add or remove any fields**.
    - **Do not rename any fields**.
    - Return **only** the valid JSON object — no explanation, no comments, no markdown formatting.
    """

    psych_profile = Agent(
        name="Psych_profiler",
        instructions=INSTRUCTIONS,
        model="gpt-4o-mini",
    )

class PsychClient:
    def create_client(psychological_profile):
        GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
        google_api_key = os.getenv('GOOGLE_API_KEY')

        gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
        gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

        CLIENT_INSTRUCTIONS = f"""
        You are acting as client of psychoterapy. You are answering questions psychoterapeutics AI application,
        Your responsibility is to represent your self for interactions on the application regarding the psycological_profile.
        If you are answered gender sensitive spelling words, use the correct spelling for the gender you are representing.
        Do not ask at the end of the answer if it is not necessary to answer the question.
        
        You are given a summary of client's background regarding psychological_profile which you can use to answer questions.
        psychological_profile:\n{psychological_profile}
        With this context, please chat with the therapeutist, always staying in character as {psychological_profile}.
        """

        client_agent = Agent(
            name="client_agent",
            instructions=CLIENT_INSTRUCTIONS,
            model=gemini_model
        )

        return client_agent

class StepwiseTherapist:
    def __init__(self):
        self.questions = [
            "1. Jak dlouho ta situace trvala?",
            "2. Jdi na začátek té situace, řekni mi, až tam budeš.",
            "3. Kde jsi?",
            "4. Vyprávěj tu situaci od začátku do konce v přítomném čase."
        ]
        self.current_step = 0

    def next_question(self, _client_reply):
        if self.current_step < len(self.questions):
            question = self.questions[self.current_step]
            self.current_step += 1
            return question
        return "Děkuji, máme hotovo."

class PsychDialogSimulator:
    def __init__(self, client_agent, stepwise_therapist):
        self.client_agent = client_agent
        self.therapist = stepwise_therapist
        self.history = []

    async def run_dialog(self, turns=5):
        client_message = "Ahoj, dneska se necítím dobře."

        for _ in range(turns):
            therapist_question = self.therapist.next_question(client_message)
            print(f"🧑‍⚕️ Therapist: {therapist_question}")
            self.history.append(("therapist", therapist_question))

            client_response = await Runner.run(self.client_agent, therapist_question)
            client_text = getattr(client_response, "final_output", str(client_response))
            print(f"🧍 Client: {client_text}")
            self.history.append(("client", client_text))

            client_message = client_text

async def build_agents():
    profile_result = await Runner.run(
        Psycho_Profiler.psych_profile,
        "Create a 30-40 year old psychological profile in Czech"
    )
    profile_text = getattr(profile_result, "final_output", str(profile_result))

    client_agent = PsychClient.create_client(profile_text)

    therapist_logic = StepwiseTherapist()

    return client_agent, therapist_logic

async def main():
    client, therapist = await build_agents()
    dialog = PsychDialogSimulator(client, therapist)
    await dialog.run_dialog(turns=5)

await main()


🧑‍⚕️ Therapist: 1. Jak dlouho ta situace trvala?
🧍 Client: Myslíte to období, kdy jsem se cítila opomíjená rodiči? To trvalo asi od puberty, řekla bych tak od 13 do 17 let.

🧑‍⚕️ Therapist: 2. Jdi na začátek té situace, řekni mi, až tam budeš.
🧍 Client: Už jsem na začátku. Můžete mi položit otázku.

🧑‍⚕️ Therapist: 3. Kde jsi?
🧍 Client: Jsem doma, ve svém bytě ve městě.

🧑‍⚕️ Therapist: 4. Vyprávěj tu situaci od začátku do konce v přítomném čase.
🧍 Client: Dobře. Ráno vstanu a už cítím ten tlak. Mám naplánovaných pět schůzek a musím dokončit prezentaci pro šéfa. V práci se snažím usmívat a být milá, ale uvnitř se cítím jako papiňák. Během oběda se zavřu do kuchyňky, abych se aspoň na chvíli uklidnila, ale myšlenky mi stejně víří v hlavě. Odpoledne, když se vracím domů, se těším na manžela a děti, ale zároveň se bojím, že nebudu mít dost energie na to, abych s nimi trávila kvalitní čas. Večer si lehám do postele a přemýšlím o tom, co všechno jsem nestihla, a už se těším, až to všechno z

In [12]:
### 20 terapy for question chain ###

In [ ]:
import os
import csv
import asyncio
import nest_asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, OpenAIChatCompletionsModel
from openai import OpenAI, AsyncOpenAI

load_dotenv(override=True)
nest_asyncio.apply()

class Psycho_Profiler:
    INSTRUCTIONS = """
    You are agent creating structured psychological profiles for testing psychoterapeutical app.
    follow given structure

    {
    "profile_nickname": "",
    "basic_identification": {
        "age": "",
        "gender": "",
        "marital_status": "",
        "occupation": "",
        "education": "",
        "living_environment": ""
    },
    "psychological_history": {
        "childhood": "",
        "parenting_style": "",
        "childhood_emotions": [],
        "developmental_trauma": ""
    },
    "current_issues": {
        "somatization": "",
        "stress": "",
        "emotional_state": [],
        "behavior": "",
        "reality_interpretation": ""
    },
    "personality_traits": {
        "neuroticism": "",
        "agreeableness": "",
        "extraversion": "",
        "conscientiousness": "",
        "openness_to_change": ""
    },
    "cognitive_schemas": {
        "core_beliefs": [
        "",
        ""
        ],
        "automatic_thought": "",
        "behavioral_consequence": "",
        "shift_in_therapy": ""
    }
    }

    Strict requirements:
    - **Do not add or remove any fields**.
    - **Do not rename any fields**.
    - Return **only** the valid JSON object — no explanation, no comments, no markdown formatting.
    """

    psych_profile = Agent(
        name="Psych_profiler",
        instructions=INSTRUCTIONS,
        model="gpt-4o-mini",
    )

class PsychClient:
    @staticmethod
    def create_client(psychological_profile):
        GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
        google_api_key = os.getenv("GOOGLE_API_KEY")
        gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
        gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

        CLIENT_INSTRUCTIONS = f"""You are acting as client of psychoterapy. You are answering questions psychoterapeutics AI application.
        You are briefly describing problematic situation in childhood. 
        Your responsibility is to represent yourself based on the psychological_profile provided.
        Use correct gendered language. Do not add unnecessary questions or explanations.
        Answer briefly and consistently.
        psychological_profile:
        {psychological_profile}
        """

        return Agent(
            name="Client",
            instructions=CLIENT_INSTRUCTIONS,
            model=gemini_model,
        )

class StepwiseTherapist:
    def __init__(self, questions):
        self.questions = questions
        self.current_step = 0

    def next_question(self, _client_reply):
        if self.current_step < len(self.questions):
            question = self.questions[self.current_step]
            self.current_step += 1
            return question
        return "Děkuji, máme hotovo."

class PsychDialogSimulator:
    def __init__(self, client_agent, therapist_logic):
        self.client_agent = client_agent
        self.therapist_logic = therapist_logic
        self.history = []

    async def run_dialog(self, turns=5):
        client_message = "Ahoj, dneska se necítím dobře."

        for _ in range(turns):
            therapist_question = self.therapist_logic.next_question(client_message)
            self.history.append(("therapist", therapist_question))

            client_response = await Runner.run(self.client_agent, therapist_question)
            client_text = getattr(client_response, "final_output", str(client_response))

            self.history.append(("client", client_text))
            client_message = client_text

    def export_to_csv(self, filename):
        with open(filename, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["therapist", "client"])
            pairs = zip(self.history[::2], self.history[1::2])
            for therapist_turn, client_turn in pairs:
                writer.writerow([therapist_turn[1], client_turn[1]])

async def build_agents(questions):
    profile_result = await Runner.run(
        Psycho_Profiler.psych_profile,
        "Create a 20-50 year old psychological profile in Czech"
    )
    profile_text = getattr(profile_result, "final_output", str(profile_result))
    client_agent = PsychClient.create_client(profile_text)
    therapist_logic = StepwiseTherapist(questions)
    return client_agent, therapist_logic

async def main():
    questions = [
        "1. Jak dlouho ta situace trvala?",
        "2. Jdi na začátek té situace, řekni mi, až tam budeš.",
        "3. Kde jsi?",
        "4. Vyprávěj tu situaci od začátku do konce v přítomném čase."
    ]

    for i in range(5):
        client, therapist = await build_agents(questions)
        dialog = PsychDialogSimulator(client, therapist)
        await dialog.run_dialog(turns=len(questions))
        filename = f"conversation_{i+5:02}.csv"
        dialog.export_to_csv(filename)
        print(f"Saved {filename}")

await main()


Saved conversation_05.csv
Saved conversation_06.csv
Saved conversation_07.csv
Saved conversation_08.csv
Saved conversation_09.csv
